<a href="https://colab.research.google.com/github/GaneshiUmayangana/GaneshiUmayangana-Deep-Learning-Mini-Project-02/blob/main/Deep_Learning_Mini_Project_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Embedding,LSTM
from keras.metrics import RootMeanSquaredError
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

#Setting up the data folder into th Colab notebook

In [19]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
os.chdir("/content/drive/My Drive/Colab_Data_Files/DeepLearning_MiniProject2/")
os.getcwd()

'/content/drive/My Drive/Colab_Data_Files/DeepLearning_MiniProject2'

#Read data file

In [21]:
df = pd.read_csv("CSE.csv")

In [22]:
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1997-07-01,798.500000,798.500000,798.500000,798.500000,798.500000,0.0
1,1997-07-02,805.299988,805.299988,805.299988,805.299988,805.299988,0.0
2,1997-07-03,810.799988,810.799988,810.799988,810.799988,810.799988,0.0
3,1997-07-07,828.200012,828.200012,828.200012,828.200012,828.200012,0.0
4,1997-07-08,821.400024,821.400024,821.400024,821.400024,821.400024,0.0
5,1997-07-09,816.700012,816.700012,816.700012,816.700012,816.700012,0.0
6,1997-07-10,820.700012,820.700012,820.700012,820.700012,820.700012,0.0
7,1997-07-11,823.400024,823.400024,823.400024,823.400024,823.400024,0.0
8,1997-07-14,821.900024,821.900024,821.900024,821.900024,821.900024,0.0
9,1997-07-15,NaN,NaN,NaN,NaN,NaN,NaN


#Create X and Y

In [23]:
x = np.array(df["Open"].values)
y = np.array(df["Adj Close"].values)

#Reading the data file

Check the missing value

In [24]:
# Check for missing values and fill them with mean
missing_values_x = df["Open"].isnull().sum()
missing_values_y = df["Adj Close"].isnull().sum()

print(f"Missing values in 'Open': {missing_values_x}")
print(f"Missing values in 'Adj Close': {missing_values_y}")

if missing_values_x > 0 or missing_values_y > 0:
    df["Open"].fillna(df["Open"].mean(), inplace=True)
    df["Adj Close"].fillna(df["Adj Close"].mean(), inplace=True)

# Check missing values again
missing_values_x = df["Open"].isnull().sum()
missing_values_y = df["Adj Close"].isnull().sum()

print(f"Missing values in 'Open': {missing_values_x}")
print(f"Missing values in 'Adj Close': {missing_values_y}")


Missing values in 'Open': 439
Missing values in 'Adj Close': 439
Missing values in 'Open': 0
Missing values in 'Adj Close': 0


In [25]:
# Extract features and target
x = np.array(df["Open"].values).reshape(-1, 1)
y = np.array(df["Adj Close"].values)

# Normalize the data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()


In [29]:
x_filtered = []

for review in x:
    if isinstance(review, str):  # Check if the element is a string
        review = review.lower()

        # Removing punctuations from the sentence
        for i in review:
            punc = ''' !{}-[]{};:"\,<>./?@#$%^_~'''
            if i in punc:
                review = review.replace(i, " ")

        x_filtered.append(review)
    else:
        # Handle non-string elements (e.g., numpy.float64) as needed
        x_filtered.append(str(review))

# Display the filtered list
#print(x_filtered)


#Split the data file

In [26]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#Model Creation and Calculate RMSE values

In [27]:
# Reshape input data to meet the requirements of the LSTM model
x_train_reshaped = x_train.reshape((x_train.shape[0], 1, 1))
x_test_reshaped = x_test.reshape((x_test.shape[0], 1, 1))

# Define the model
model = Sequential()

# Add an LSTM layer with 50 units and 'relu' activation
model.add(LSTM(50, activation='relu', input_shape=(1, 1)))

# Add a Dense layer for the output with 1 unit (assuming it's a regression task)
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae', RootMeanSquaredError()])  # Add RMSE as a metric

# Train the model
history = model.fit(x_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(x_test_reshaped, y_test))

# Evaluate the model on the test set
loss, mae, rmse = model.evaluate(x_test_reshaped, y_test)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")
print(f"Test RMSE: {rmse}")

# Print the training history (loss, MAE, and validation loss over epochs)
print("Training Loss:", history.history['loss'])
#print("Training MAE:", history.history['mean_absolute_error'])
print("Validation Loss:", history.history['val_loss'])

Epoch 1/10
149/149 [==============================] - 2s 5ms/step - loss: 10715300.0000 - mae: 2667.8047 - root_mean_squared_error: 3273.4233 - val_loss: 4063163.2500 - val_mae: 1662.7440 - val_root_mean_squared_error: 2015.7290
Epoch 2/10
149/149 [==============================] - 1s 3ms/step - loss: 1388763.1250 - mae: 877.8488 - root_mean_squared_error: 1178.4579 - val_loss: 93962.5781 - val_mae: 251.7499 - val_root_mean_squared_error: 306.5332
Epoch 3/10
149/149 [==============================] - 1s 3ms/step - loss: 18044.6387 - mae: 86.4539 - root_mean_squared_error: 134.3303 - val_loss: 1067.8472 - val_mae: 18.8368 - val_root_mean_squared_error: 32.6779
Epoch 4/10
149/149 [==============================] - 1s 3ms/step - loss: 1177.4055 - mae: 18.3684 - root_mean_squared_error: 34.3133 - val_loss: 1005.6971 - val_mae: 17.5881 - val_root_mean_squared_error: 31.7127
Epoch 5/10
149/149 [==============================] - 0s 3ms/step - loss: 1168.4489 - mae: 18.2139 - root_mean_squared

#Predict Stock Price

In [28]:
# Use the trained model to make predictions on the test set
y_pred = model.predict(x_test_reshaped)

# If you reshaped the input data, flatten the predictions for easier comparison
y_pred = y_pred.flatten()

# Create a DataFrame to compare the actual and predicted values
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results)

38/38 [==============================] - 0s 2ms/step
           Actual    Predicted
0     5985.450195  5980.903809
1     3617.430232  3616.937012
2     1807.170044  1807.488770
3     6519.229980  6522.627930
4      780.390015   781.854187
...           ...          ...
1185  2997.070068  3024.234375
1186   421.540009   419.564941
1187  6401.319824  6353.847656
1188  7403.379883  7419.638672
1189  3617.430232  3616.937012

[1190 rows x 2 columns]
